In [45]:
import numpy as np
import os
import shutil

In [46]:
def rotation_matrix(u,theta):
    """
    description:
    generates a 3x3 rotation matrix given an axis and angle
    
    inputs: 
    u = unit vector for rotation axis given as a list [ux, uy, uz]
    theta = angle for rotation in radians
    
    output:
    Rotation matrix as a 3x3 numpy array
    """
    ux = u[0]
    uy = u[1]
    uz = u[2]
    R = np.zeros((3,3))
    R[0,0] = np.cos(theta)+ux**2*(1-np.cos(theta))
    R[0,1] = ux*uy*(1-np.cos(theta))-uz*np.sin(theta)
    R[0,2] = ux*uz*(1-np.cos(theta))+uy*np.sin(theta)
    R[1,0] = uy*ux*(1-np.cos(theta))+uz*np.sin(theta)
    R[1,1] = np.cos(theta)+uy**2*(1-np.cos(theta))
    R[1,2] = uy*uz*(1-np.cos(theta))-ux*np.sin(theta)
    R[2,0] = uz*ux*(1-np.cos(theta))-uy*np.sin(theta)
    R[2,1] = uz*uy*(1-np.cos(theta))+ux*np.sin(theta)
    R[2,2] = np.cos(theta)+uz**2*(1-np.cos(theta))
    
    return R

In [56]:
# enter the path for your poscar (you can create this by exporting a vesta CIF)
# make sure the poscar is generated with fractional (direct) coordinates
poscar_path = '/Users/Thomas2/Desktop/2D_diffraction-main/Casey_N2200/N2000_UofA_MD_DFT.vasp'
save_name = 'N2200_UA_faceon.vasp' # this will be how the saved poscar is named

#copy the existing poscar file to a new poscar with specified name
working_dir = os.path.dirname(poscar_path)
save_path = working_dir + '/' + save_name
shutil.copyfile(poscar_path, save_path)

# grab the unit cell vectors from the poscar
# Remember these are not necessarily perpendicular!
cell_vects = np.loadtxt(poscar_path, skiprows=2, max_rows=3)
print('old cell:'+'\n'+np.array_str(cell_vects, precision=2, suppress_small=True))
cell_a = cell_vects[0,:]
cell_b = cell_vects[1,:]
cell_c = cell_vects[2,:]

#specify which two vectors you would like to make parallel here
vect1 = np.cross(cell_a, cell_c)
vect2 = np.asarray([0,0,1])

#calculate the rotation unit axis between the two vectors
rot_axis = np.cross(vect1, vect2)
unit_rot_axis = rot_axis/np.linalg.norm(rot_axis)

#calculate the angle between the two vectors
rot_theta = np.arccos(np.dot(vect1, vect2)/
                      (np.linalg.norm(vect1)*np.linalg.norm(vect2)))

#generate a rotation matrix
rot_matrix = rotation_matrix(unit_rot_axis, rot_theta)

#rotate the old cell unit vectors
new_cell_a = np.matmul(rot_matrix, cell_a)
new_cell_b = np.matmul(rot_matrix, cell_b)
new_cell_c = np.matmul(rot_matrix, cell_c)

#generate a new cell matrix
new_cell = np.stack((new_cell_a, new_cell_b, new_cell_c))
print('new cell:'+'\n'+np.array_str(new_cell, precision=2, suppress_small=True))

# Write the new cell to the poscar 
# Read the existing text file
with open(save_path, 'r') as file:
    lines = file.readlines()
# Split the array string into separate lines
array_lines = array_string.split('\n')
# Replace lines 3,4,and 5
lines[2] = '\t' + '\t'.join(map(lambda x: f"{x:.10f}", new_cell[0,:])) + '\n'
lines[3] = '\t' + '\t'.join(map(lambda x: f"{x:.10f}", new_cell[1,:])) + '\n'
lines[4] = '\t' + '\t'.join(map(lambda x: f"{x:.10f}", new_cell[2,:])) + '\n'
# Write the modified content back to the file
with open(save_path, 'w') as file:
    file.writelines(lines)
file.close()

old cell:
[[14.31  0.    0.  ]
 [ 2.44  4.27  0.  ]
 [12.21 10.17 24.64]]
new cell:
[[14.31  0.    0.  ]
 [ 2.44  1.63 -3.94]
 [12.21 26.66  0.  ]]
